In [ ]:
import pandas as pd
import warnings
import numpy as np
import statsmodels as sm

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

### Note: data merge rules

selected_merge_dta.id == selected_NBS_dta_complete.id
selected_merge_dta.year == selected_NBS_dta_complete.year
selected_merge_dta.idc == selected_CUS_200x.idc200x
selected_merge_dta.year == selected_CUS_2000.year

In [ ]:
# read data
merge_dta = pd.read_stata('nbsid_cusidc.dta') # joint data
NBS_dta_complete = pd.read_stata('qyfinal.dta') # Chinese Industrial Enterprises Database

In [ ]:
# process joint data
merge_dta['regime_name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in merge_dta['regime_name'].tolist()])
merge_dta['name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in merge_dta['name'].tolist()])
merge_dta.idc = merge_dta.idc.fillna(0).astype(int)
selected_merge_dta = merge_dta[['id','idc','year','name']] # remain such variables and drop others

In [ ]:
# process NBS data
NBS_dta_complete['frdwdm'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in NBS_dta_complete['frdwdm'].tolist()])
NBS_dta_complete.id = NBS_dta_complete.id.astype(int)

selected_NBS_dta_complete = NBS_dta_complete[['id', 'year', 'frdwdm', 'youbian', 'setup_year', 
                                              'setup_month', 'icode', 'konggu', 'totaloutput',
                                              'newproduct_output', 'total_sale', 'export', 
                                              'aemployment', 'valueadded', 'longrun_invest', 'totalfixed_asset', 
                                              'fixed_asset_op', 'depreciate', 'depreciate_tyear', 'fixedasset_net', 
                                              'total_asset', 'owner_rights', 'paicl_up_capital', 'so_capital', 
                                              'co_capital', 'lp_capital', 'p_capital', 'gat_capital', 'foreign_capital',
                                              'mainbusiness_revenue', 'mainbusiness_cost', 'finance_expense', 'fe_interest',
                                              'total_profit', 'advertise_expense', 'wage_payable', 'wp_mainbusiness', 
                                              'totalintermediate_input']]
# remain such variables and drop others, details:
# remain variables: idc2000(公司代码); year(年份); value_ex(出口额); value_im(进口额); 
# regime(贸易类型代码); regime_name(贸易类型); origin(原产国/起运国代码); originn(原产国/起运国名称); 
# code_city(省市代码); code_cityn(省市名称); ownershipc(企业类型代码); ownershipc_name(企业类型名称); 
# transport(运输方式代码); transport_name(运输方式名称); price(单价); quantity_ex(出口量); quantity_im(进口量); 
# ex(是否出口:1yes0no); im(是否进口:1yes0no);
# drop variables: name(公司名); hs8(hs编码); EX(是否出口:0yes1no); unit(产品单位代码); unit_name(产品单位名称); 
# address(地址), telephone(电话), fax(传真), postcode(邮编), email, linkman(法人); product_name(产品名称); 
# custom_code(海关代码); custom_name(海关名称); pass_country(途经国代码); pass_country_name(途经国名称); firm_code; totalex;


# first stage merge: NBS + joint data
intermediate_dta = pd.merge(selected_merge_dta, selected_NBS_dta_complete, how='left', on=['year','id'])

In [ ]:
# read data
CUS_2000 = pd.read_stata('c2000.dta') 
# Chinese Custom Database 2000, in stata file formant (for following years we transform them into CSV formant since the limitation of RAM)

# process CUS data
CUS_2000['name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['name'].tolist()])
CUS_2000['regime_name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['regime_name'].tolist()])
CUS_2000['originn'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['originn'].tolist()])
CUS_2000['code_cityn'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['code_cityn'].tolist()])
CUS_2000['ownershipc_name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['ownershipc_name'].tolist()])
CUS_2000['transport_name'] = pd.Series([x.encode('latin-1').decode('gb18030') for x in CUS_2000['transport_name'].tolist()])

CUS_2000.year = 2000
CUS_2000.idc2000 = CUS_2000.idc2000.astype(int)
selected_CUS_2000 = CUS_2000[['idc2000', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownershipc_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]
# remian such variables and drop others

# value_ex/value_im = quantity_ex/quantity_im * price
# year is constant for each CUS data file

selected_CUS_2000.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']
# rename columns for match with other data in later stage

# read data
CUS_2001 = pd.read_csv('c2001.csv')
CUS_2002 = pd.read_csv('c2002.csv')
CUS_2003 = pd.read_csv('c2003.csv')
CUS_2004 = pd.read_csv('c2004.csv')
CUS_2005 = pd.read_csv('c2005.csv')
CUS_2006 = pd.read_csv('c2006.csv')
# Chinese Custom Database 2001 - 2006, in CSV format, and basically the same processing procedure

CUS_2001['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2001['regime_name'].tolist()])
CUS_2001['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2001['originn'].tolist()])
CUS_2001['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2001['code_cityn'].tolist()])
CUS_2001['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2001['ownership_name'].tolist()])
CUS_2001['transport_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2001['transport_name'].tolist()])
CUS_2001.year = 2001
CUS_2001.idc2001 = CUS_2001.idc2001.astype(int)   

selected_CUS_2001 = CUS_2001[['idc2001', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2001.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

CUS_2002['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2002['regime_name'].tolist()])
CUS_2002['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2002['originn'].tolist()])
CUS_2002['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2002['code_cityn'].tolist()])
CUS_2002['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2002['ownership_name'].tolist()])
CUS_2002['transport_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2002['transport_name'].tolist()])
CUS_2002.year = 2002
CUS_2002.idc2002 = CUS_2002.idc2002.astype(int)   

selected_CUS_2002 = CUS_2002[['idc2002', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2002.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

CUS_2003['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2003['regime_name'].tolist()])
CUS_2003['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2003['originn'].tolist()])
CUS_2003['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2003['code_cityn'].tolist()])
CUS_2003['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2003['ownership_name'].tolist()])
CUS_2003['transport_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2003['transport_name'].tolist()])
CUS_2003.year = 2003
CUS_2003.idc2003 = CUS_2003.idc2003.astype(int)   

selected_CUS_2003 = CUS_2003[['idc2003', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2003.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

CUS_2004['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2004['regime_name'].tolist()])
CUS_2004['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2004['originn'].tolist()])
CUS_2004['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2004['code_cityn'].tolist()])
CUS_2004['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2004['ownership_name'].tolist()])
CUS_2004['transport_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2004['transport_name'].tolist()])
CUS_2004.year = 2004
CUS_2004.idc2004 = CUS_2004.idc2004.astype(int)   

selected_CUS_2004 = CUS_2004[['idc2004', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2004.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

CUS_2005['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2005['regime_name'].tolist()])
CUS_2005['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2005['originn'].tolist()])
CUS_2005['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2005['code_cityn'].tolist()])
CUS_2005['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2005['ownership_name'].tolist()])
CUS_2005['transport_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2005['transport_name'].tolist()])
CUS_2005.year = 2005
CUS_2005.idc2005 = CUS_2005.idc2005.astype(int)   

selected_CUS_2005 = CUS_2005[['idc2005', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'transport_name', 'price', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2005.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'transport_name', 'price', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

CUS_2006['regime_name'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2006['regime_name'].tolist()])
CUS_2006['originn'] = pd.Series([str(x).encode('latin-1').decode('gb18030') for x in CUS_2006['originn'].tolist()])
CUS_2006['code_cityn'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2006['code_cityn'].tolist()])
CUS_2006['ownership_name'] = pd.Series([str(x).encode('cp1252').decode('gb18030') for x in CUS_2006['ownership_name'].tolist()])
CUS_2006.year = 2006
CUS_2006.idc2006 = CUS_2006.idc2006.astype(int)   

selected_CUS_2006 = CUS_2006[['idc2006', 'year', 'value_ex', 'value_im', 
                              'regime', 'regime_name', 'origin', 'originn', 
                              'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                              'transport', 'quantity_ex', 
                              'quantity_im', 'ex', 'im']]

selected_CUS_2006.columns = ['idc', 'year', 'value_ex', 'value_im', 
                             'regime', 'regime_name', 'origin', 'originn', 
                             'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                             'transport', 'quantity_ex', 
                             'quantity_im', 'ex', 'im']

In [ ]:
# add columns, we do this just for data in 2006 because they are the same for data in other years
d = {1.0: u'航空运输', 2.0: u'江海运输', 3.0: u'铁路运输', 4.0: u'汽车运输', 5.0: u'航空运输', 6.0: u'邮件运输'}
selected_CUS_2006['transport_name'] = selected_CUS_2006['transport'].map(d)

selected_CUS_2006['price'] = ''
selected_CUS_2006['price'][selected_CUS_2006.value_ex > 0] = selected_CUS_2006.value_ex / selected_CUS_2006.quantity_ex
selected_CUS_2006['price'][selected_CUS_2006.value_im > 0] = selected_CUS_2006.value_im / selected_CUS_2006.quantity_im

In [ ]:
# concatenate Chinese Custom Database
CUS_final = pd.concat([selected_CUS_2000, selected_CUS_2001, selected_CUS_2002, selected_CUS_2003, 
                       selected_CUS_2004, selected_CUS_2005, selected_CUS_2006], axis=0, ignore_index=True)

CUS_final = CUS_final[['idc', 'year', 'value_ex', 'value_im', 
                       'regime', 'regime_name', 'origin', 'originn', 
                       'code_city', 'code_cityn', 'ownershipc', 'ownership_name',
                       'transport', 'transport_name', 'price', 'quantity_ex', 
                       'quantity_im', 'ex', 'im']]

In [ ]:
# second stage merge
final_dta = pd.merge(intermediate_dta, CUS_final, how='left', on=['idc', 'year'])

final_dta_sorted = final_dta.sort_values(by=['id', 'year'])
final_dta_sorted['ex'].fillna(0, inplace=True)
final_dta_sorted['im'].fillna(0, inplace=True)

In [1]:
dta_droped1 = final_dta_sorted[final_dta_sorted.value_ex.notnull()] 
# drop value_ex, value_im, quantity_ex, quantity_im, price with NaNs

dta_droped2 = dta_droped1[dta_droped1.totaloutput.notnull()] 
# drop totaloutput, total_sale, export with NaNs

dic = {u'其它': 39.0, u'保税仓库进出境货物': 33.0, u'出口加工区进口设备': 35.0}
dta_droped2.regime = dta_droped2.regime.fillna(dta_droped2.regime_name.map(dic))
# fill regime with the mapping rule we define in dic

dta_droped3 = dta_droped2.drop(['newproduct_output', 'advertise_expense', 'depreciate_tyear', 'Unnamed: 0', 'idc'], axis=1)
# drop variables newproduct_output; advertise_expense

dta_droped4 = dta_droped3.dropna(subset=['name', 'youbian', 'ownershipc', 'transport', 'code_cityn', 'code_city', 
                                         'valueadded', 'origin', 'setup_month', 'setup_year', ], how='any')
# these columns(variables) can't have NaNs

dta_droped4['exchange_rate'] = ''
# initiate the exchange rate column

# add exchange rate data by hand, command + c and command + v, tedious work
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2000)] = 0.12080
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2001)] = 0.12086
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2002)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2003)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2004)] = 0.12097
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2005)] = 0.12221
dta_droped4['exchange_rate'][(dta_droped4.originn == '美国') & (dta_droped4.year == 2006)] = 0.12557

dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2000)] = 0.20840
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2001)] = 0.23369
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2002)] = 0.22271
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2003)] = 0.18650
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2004)] = 0.16450
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2005)] = 0.16040
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳大利亚') & (dta_droped4.year == 2006)] = 0.16679

dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2000)] = 1.14338
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2001)] = 1.12959
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2002)] = 1.17124
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2003)] = 1.30710
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2004)] = 1.36812
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2005)] = 1.33245
dta_droped4['exchange_rate'][(dta_droped4.originn == '墨西哥') & (dta_droped4.year == 2006)] = 1.37068

dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2000)] = 136.63969
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2001)] = 156.08650
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2002)] = 151.18033
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2003)] = 144.49267
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2004)] = 139.22766
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2005)] = 125.59191
dta_droped4['exchange_rate'][(dta_droped4.originn == '韩国') & (dta_droped4.year == 2006)] = 121.77112

dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '德国') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2000)] = 37.16006
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2001)] = 40.26550
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2002)] = 45.04910
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2003)] = 49.50214
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2004)] = 54.13288
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2005)] = 58.97424
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥斯达黎加') & (dta_droped4.year == 2006)] = 66.27414

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2000)] = 0.22114
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2001)] = 0.28751
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2002)] = 0.35891
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2003)] = 0.37701
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2004)] = 0.35400
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2005)] = 0.29737
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴西') & (dta_droped4.year == 2006)] = 0.27371

dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2000)] = 13.02913
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2001)] = 14.69087
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2002)] = 15.14706
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2003)] = 14.02900
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2004)] = 13.08612
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2005)] = 13.46376
dta_droped4['exchange_rate'][(dta_droped4.originn == '日本') & (dta_droped4.year == 2006)] = 14.60877

dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2000)] = 3.77642
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2001)] = 4.10740
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2002)] = 4.18313
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2003)] = 4.17082
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2004)] = 4.04881
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2005)] = 3.93609
dta_droped4['exchange_rate'][(dta_droped4.originn == '台湾省') & (dta_droped4.year == 2006)] = 4.08826

dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '比利时') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '意大利') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2000)] = 0.94128
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2001)] = 0.94265
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2002)] = 0.94350
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2003)] = 0.94211
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2004)] = 0.94228
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2005)] = 0.95051
dta_droped4['exchange_rate'][(dta_droped4.originn == '香港') & (dta_droped4.year == 2006)] = 0.97553

dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '法国') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2000)] = 0.07983
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2001)] = 0.08392
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2002)] = 0.08061
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2003)] = 0.07406
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2004)] = 0.06606
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2005)] = 0.06724
dta_droped4['exchange_rate'][(dta_droped4.originn == '英国') & (dta_droped4.year == 2006)] = 0.06823

dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2000)] = 0.52580
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2001)] = 0.49537
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2002)] = 0.49371
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2003)] = 0.47038
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2004)] = 0.44182
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2005)] = 0.39579
dta_droped4['exchange_rate'][(dta_droped4.originn == '波兰') & (dta_droped4.year == 2006)] = 0.39013

dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2000)] = 0.08218
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2001)] = 0.16812
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2002)] = 0.19507
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2003)] = 0.17532
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2004)] = 0.17447
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2005)] = 0.16645
dta_droped4['exchange_rate'][(dta_droped4.originn == '土耳其') & (dta_droped4.year == 2006)] = 0.16795

dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '西班牙') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2000)] = 0.44369
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2001)] = 0.44396
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2002)] = 0.44433
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2003)] = 0.44432
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2004)] = 0.44440
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2005)] = 0.44892
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿拉伯联合酋长国') & (dta_droped4.year == 2006)] = 0.45713

dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2000)] = 0.45904
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2001)] = 0.45938
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2002)] = 0.45999
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2003)] = 0.45979
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2004)] = 0.45985
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2005)] = 0.46298
dta_droped4['exchange_rate'][(dta_droped4.originn == '马来西亚') & (dta_droped4.year == 2006)] = 0.46306

dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2000)] = 5.43006
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2001)] = 5.70768
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2002)] = 5.88844
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2003)] = 5.64407
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2004)] = 5.48483
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2005)] = 5.39204
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度') & (dta_droped4.year == 2006)] = 5.54170

dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2000)] = 0.20832
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2001)] = 0.21661
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2002)] = 0.21662
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2003)] = 0.21081
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2004)] = 0.20454
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2005)] = 0.20345
dta_droped4['exchange_rate'][(dta_droped4.originn == '新加坡') & (dta_droped4.year == 2006)] = 0.20191

dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '荷兰') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2000)] = 0.12075
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2001)] = 0.14983
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2002)] = 0.41208
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2003)] = 0.35489
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2004)] = 0.35822
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2005)] = 0.36524
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿根廷') & (dta_droped4.year == 2006)] = 0.39319

dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2000)] = 0.08605
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2001)] = 0.08626
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2002)] = 0.08625
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2003)] = 0.08630
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2004)] = 0.08624
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2005)] = 0.08757
dta_droped4['exchange_rate'][(dta_droped4.originn == '约旦') & (dta_droped4.year == 2006)] = 0.08915

dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2000)] = 67.05529
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2001)] = 81.24263
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2002)] = 86.51439
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2003)] = 80.87983
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2004)] = 74.68273
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2005)] = 67.51761
dta_droped4['exchange_rate'][(dta_droped4.originn == '智利') & (dta_droped4.year == 2006)] = 66.50590

dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2000)] = 13.14983
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2001)] = 14.29043
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2002)] = 15.60368
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2003)] = 16.28025
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2004)] = 15.91334
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2005)] = 16.29715
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼日利亚') & (dta_droped4.year == 2006)] = 16.49839

dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2000)] = 3028.59381
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2001)] = 3027.17011
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2002)] = 3094.42323
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2003)] = 3126.81533
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2004)] = 3119.96000
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2005)] = 3176.95189
dta_droped4['exchange_rate'][(dta_droped4.originn == '厄瓜多尔') & (dta_droped4.year == 2006)] = 3222.64700

dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2000)] = 0.18166
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2001)] = 0.18926
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2002)] = 0.18746
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2003)] = 0.16368
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2004)] = 0.15469
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2005)] = 0.14689
dta_droped4['exchange_rate'][(dta_droped4.originn == '加拿大') & (dta_droped4.year == 2006)] = 0.14331

dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2000)] = 5.02423
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2001)] = 5.3994511
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2002)] = 5.18198
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2003)] = 4.92483
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2004)] = 4.85717
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2005)] = 4.98172
dta_droped4['exchange_rate'][(dta_droped4.originn == '泰国') & (dta_droped4.year == 2006)] = 4.76427

dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2000)] = 1088.97140
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2001)] = 1256.20392
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2002)] = 1092.22452
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2003)] = 1026.71147
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2004)] = 1106.38082
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2005)] = 1197.52082
dta_droped4['exchange_rate'][(dta_droped4.originn == '印度尼西亚') & (dta_droped4.year == 2006)] = 1117.81500

dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2000)] = 3.40506
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2001)] = 3.59649
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2002)] = 3.82208
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2003)] = 3.62202
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2004)] = 3.46255
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2005)] = 3.49016
dta_droped4['exchange_rate'][(dta_droped4.originn == '俄罗斯联邦') & (dta_droped4.year == 2006)] = 3.44820

dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2000)] = 1728.79277
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2001)] = 1854.42419
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2002)] = 1932.57674
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2003)] = 1949.44697
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2004)] = 1949.18792
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2005)] = 1968.65110
dta_droped4['exchange_rate'][(dta_droped4.originn == '越南') & (dta_droped4.year == 2006)] = 2051.45233

dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2000)] = 0.20494
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2001)] = 0.20457
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2002)] = 0.17892
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2003)] = 0.15938
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2004)] = 0.14811
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2005)] = 0.15682
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞士') & (dta_droped4.year == 2006)] = 0.16071

dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2000)] = 5.61849
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2001)] = 6.24225
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2002)] = 6.34070
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2003)] = 6.62794
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2004)] = 6.76663
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2005)] = 6.68631
dta_droped4['exchange_rate'][(dta_droped4.originn == '菲律宾') & (dta_droped4.year == 2006)] = 6.43212

dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2000)] = 0.88441
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2001)] = 1.15308
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2002)] = 1.17845
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2003)] = 0.86844
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2004)] = 0.75820
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2005)] = 0.78965
dta_droped4['exchange_rate'][(dta_droped4.originn == '南非') & (dta_droped4.year == 2006)] = 0.76304

dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2000)] = 1.14144
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2001)] = 1.27044
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2002)] = 1.11962
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2003)] = 0.94141
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2004)] = 0.87627
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2005)] = 0.94588
dta_droped4['exchange_rate'][(dta_droped4.originn == '瑞典') & (dta_droped4.year == 2006)] = 0.95325

dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2000)] = 0.45308
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2001)] = 0.45328
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2002)] = 0.45368
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2003)] = 0.45368
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2004)] = 0.45373
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2005)] = 0.45836
dta_droped4['exchange_rate'][(dta_droped4.originn == '沙特阿拉伯') & (dta_droped4.year == 2006)] = 0.47092

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2000)] = 6.51606
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2001)] = 7.56930
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2002)] = 7.53153
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2003)] = 7.24492
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2004)] = 7.26001
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2005)] = 7.30023
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴基斯坦') & (dta_droped4.year == 2006)] = 7.56519

dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2000)] = 0.26600
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2001)] = 0.28760
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2002)] = 0.26148
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2003)] = 0.20841
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2004)] = 0.18258
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2005)] = 0.17364
dta_droped4['exchange_rate'][(dta_droped4.originn == '新西兰') & (dta_droped4.year == 2006)] = 0.19370

dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2000)] = 0.49358
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2001)] = 0.50888
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2002)] = 0.57315
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2003)] = 0.55074
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2004)] = 0.54333
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2005)] = 0.54967
dta_droped4['exchange_rate'][(dta_droped4.originn == '以色列') & (dta_droped4.year == 2006)] = 0.56071

dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2000)] = 0.97703
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2001)] = 1.00607
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2002)] = 0.95379
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2003)] = 0.79582
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2004)] = 0.72466
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2005)] = 0.73297
dta_droped4['exchange_rate'][(dta_droped4.originn == '丹麦') & (dta_droped4.year == 2006)] = 0.74629

dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '芬兰') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '希腊') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2000)] = 0.96763
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2001)] = 0.98039
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2002)] = 1.00821
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2003)] = 1.00200
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2004)] = 0.99744
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2005)] = 1.01766
dta_droped4['exchange_rate'][(dta_droped4.originn == '澳门') & (dta_droped4.year == 2006)] = 1.04521

dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2000)] = 211.99713
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2001)] = 212.10363
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2002)] = 833.42308
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2003)] = 955.58953
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2004)] = 955.68078
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2005)] = 1012.89746
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊朗') & (dta_droped4.year == 2006)] = 1191.99310

dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2000)] = 6.32267
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2001)] = 6.86131
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2002)] = 7.21317
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2003)] = 7.26497
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2004)] = 7.36510
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2005)] = 7.90204
dta_droped4['exchange_rate'][(dta_droped4.originn == '孟加拉国') & (dta_droped4.year == 2006)] = 8.82666

dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2000)] = 0.42841
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2001)] = 0.49067
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2002)] = 0.56342
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2003)] = 0.71521
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2004)] = 0.75531
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2005)] = 0.71277
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃及') & (dta_droped4.year == 2006)] = 0.73052

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2000)] = 0.12080
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2001)] = 0.12086
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2002)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2003)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2004)] = 0.12097
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2005)] = 0.12221
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拿马') & (dta_droped4.year == 2006)] = 0.12557

dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2000)] = 0.66459
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2001)] = 0.65006
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2002)] = 0.66393
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2003)] = 0.66598
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2004)] = 0.66154
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2005)] = 0.63034
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌克兰') & (dta_droped4.year == 2006)] = 0.65556

dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '奥地利') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '葡萄牙') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱尔兰') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2000)] = 4.66697
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2001)] = 4.60660
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2002)] = 3.96837
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2003)] = 3.41315
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2004)] = 3.11202
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2005)] = 2.93243
dta_droped4['exchange_rate'][(dta_droped4.originn == '捷克共和国') & (dta_droped4.year == 2006)] = 2.84013

dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2000)] = 9.29130
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2001)] = 10.82966
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2002)] = 11.58572
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2003)] = 11.67863
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2004)] = 12.24700
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2005)] = 12.29441
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯里兰卡') & (dta_droped4.year == 2006)] = 13.09781

dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2000)] = 34.17267
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2001)] = 34.63607
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2002)] = 31.21883
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2003)] = 27.15605
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2004)] = 24.54840
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2005)] = 24.44402
dta_droped4['exchange_rate'][(dta_droped4.originn == '匈牙利') & (dta_droped4.year == 2006)] = 26.46812

dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2000)] = 252.77488
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2001)] = 280.87073
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2002)] = 312.08108
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2003)] = 355.37879
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2004)] = 323.76272
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2005)] = 284.91199
dta_droped4['exchange_rate'][(dta_droped4.originn == '哥伦比亚') & (dta_droped4.year == 2006)] = 304.39610

dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2000)] = 0.26189
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2001)] = 0.35471
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2002)] = 0.41247
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2003)] = 0.41242
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2004)] = 0.40459
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2005)] = 0.35883
dta_droped4['exchange_rate'][(dta_droped4.originn == '罗马尼亚') & (dta_droped4.year == 2006)] = 0.35374

dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2000)] = 182.69358
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2001)] = 184.16214
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2002)] = 189.40609
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2003)] = 187.87951
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2004)] = 187.24408
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2005)] = 185.25421
dta_droped4['exchange_rate'][(dta_droped4.originn == '黎巴嫩') & (dta_droped4.year == 2006)] = 193.70782

dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2000)] = 1.28542
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2001)] = 1.36778
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2002)] = 1.33931
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2003)] = 1.17182
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2004)] = 1.08459
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2005)] = 1.09491
dta_droped4['exchange_rate'][(dta_droped4.originn == '摩洛哥') & (dta_droped4.year == 2006)] = 1.11840

dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2000)] = 0.42151
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2001)] = 0.42915
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2002)] = 0.44329
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2003)] = 0.43582
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2004)] = 0.42434
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2005)] = 0.40437
dta_droped4['exchange_rate'][(dta_droped4.originn == '秘鲁') & (dta_droped4.year == 2006)] = 0.42238

dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2000)] = 0.03711
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2001)] = 0.03713
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2002)] = 0.03660
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2003)] = 0.03603
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2004)] = 0.03571
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2005)] = 0.03599
dta_droped4['exchange_rate'][(dta_droped4.originn == '科威特') & (dta_droped4.year == 2006)] = 0.03664

dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2000)] = 83.48509
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2001)] = 92.54210
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2002)] = 164.30938
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2003)] = 198.92696
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2004)] = 234.36205
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2005)] = 264.44200
dta_droped4['exchange_rate'][(dta_droped4.originn == '委内瑞拉') & (dta_droped4.year == 2006)] = 268.42933

dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2000)] = 0.68345
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2001)] = 0.80930
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2002)] = 0.79650
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2003)] = 0.77384
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2004)] = 0.75755
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2005)] = 0.79405
dta_droped4['exchange_rate'][(dta_droped4.originn == '缅甸') & (dta_droped4.year == 2006)] = 0.83719

dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2000)] = 7.51017
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2001)] = 6.48766
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2002)] = 6.27259
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2003)] = 5.91443
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2004)] = 6.36416
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2005)] = 6.48876
dta_droped4['exchange_rate'][(dta_droped4.originn == '叙利亚') & (dta_droped4.year == 2006)] = 6.82255

dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2000)] = 9.43284
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2001)] = 9.67675
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2002)] = 9.95291
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2003)] = 9.39959
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2004)] = 8.98560
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2005)] = 9.18203
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿尔及利亚') & (dta_droped4.year == 2006)] = 9.42657

dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2000)] = 0.07535
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2001)] = 0.07754
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2002)] = 0.07422
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2003)] = 0.06268
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2004)] = 0.05717
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2005)] = 0.05688
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞浦路斯') & (dta_droped4.year == 2006)] = 0.05768

dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2000)] = 767.06976
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2001)] = 905.19550
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2002)] = 1019.72519
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2003)] = 1050.26724
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2004)] = 1084.68992
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2005)] = 1125.97973
dta_droped4['exchange_rate'][(dta_droped4.originn == '加纳') & (dta_droped4.year == 2006)] = 1168.48550

dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2000)] = 0.93689
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2001)] = 0.97312
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2002)] = 0.97496
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2003)] = 0.99526
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2004)] = 0.97677
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2005)] = 0.94349
dta_droped4['exchange_rate'][(dta_droped4.originn == '危地马拉') & (dta_droped4.year == 2006)] = 0.99195

dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2000)] = 464.84347
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2001)] = 475.76850
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2002)] = 484.62586
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2003)] = 488.65880
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2004)] = 487.57789
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2005)] = 521.09704
dta_droped4['exchange_rate'][(dta_droped4.originn == '柬埔寨') & (dta_droped4.year == 2006)] = 520.65573

dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2000)] = 48.77224
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2001)] = 0.26639
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2002)] = 0.23840
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2003)] = 0.20106
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2004)] = 0.18847
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2005)] = 0.19790
dta_droped4['exchange_rate'][(dta_droped4.originn == '保加利亚') & (dta_droped4.year == 2006)] = 0.20088

dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2000)] = 28.07038
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2001)] = 30.30812
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2002)] = 28.31052
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2003)] = 24.65108
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2004)] = 23.40747
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2005)] = 24.24205
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛文尼亚共和国') & (dta_droped4.year == 2006)] = 24.47575

dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2000)] = 0.07408
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2001)] = 0.07614
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2002)] = 0.07279
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2003)] = 0.06720
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2004)] = 0.06468
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2005)] = 0.07052
dta_droped4['exchange_rate'][(dta_droped4.originn == '立陶宛') & (dta_droped4.year == 2006)] = 0.07116

dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2000)] = 9.37131
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2001)] = 9.53277
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2002)] = 9.53610
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2003)] = 9.21995
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2004)] = 9.61807
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2005)] = 9.18235
dta_droped4['exchange_rate'][(dta_droped4.originn == '肯尼亚') & (dta_droped4.year == 2006)] = 9.15488

dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2000)] = 1.37713
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2001)] = 1.65151
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2002)] = 2.98190
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2003)] = 3.45508
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2004)] = 3.34421
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2005)] = 2.98730
dta_droped4['exchange_rate'][(dta_droped4.originn == '乌拉圭') & (dta_droped4.year == 2006)] = 3.15600

dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2000)] = 17.34544
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2001)] = 17.98258
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2002)] = 18.40464
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2003)] = 18.03715
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2004)] = 16.73890
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2005)] = 16.44451
dta_droped4['exchange_rate'][(dta_droped4.originn == '克罗地亚共和国') & (dta_droped4.year == 2006)] = 16.36881

dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2000)] = 1.01693
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2001)] = 1.03069
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2002)] = 0.93584
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2003)] = 0.81239
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2004)] = 0.74478
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2005)] = 0.74481
dta_droped4['exchange_rate'][(dta_droped4.originn == '哈萨克斯坦') & (dta_droped4.year == 2006)] = 0.72491

dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2000)] = 0.12080
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2001)] = 0.12086
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2002)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2003)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2004)] = 0.12097
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2005)] = 0.12221
dta_droped4['exchange_rate'][(dta_droped4.originn == '波多黎各') & (dta_droped4.year == 2006)] = 0.12557

dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2000)] = 0.43982
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2001)] = 0.43871
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2002)] = 0.44080
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2003)] = 0.44064
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2004)] = 0.44069
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2005)] = 0.44503
dta_droped4['exchange_rate'][(dta_droped4.originn == '卡塔尔') & (dta_droped4.year == 2006)] = 0.45915

dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2000)] = 3.17560
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2001)] = 3.53393
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2002)] = 3.65533
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2003)] = 3.40917
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2004)] = 3.36228
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2005)] = 3.62344
dta_droped4['exchange_rate'][(dta_droped4.originn == '毛里求斯') & (dta_droped4.year == 2006)] = 4.00880

dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2000)] = 1.91717
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2001)] = 1.98581
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2002)] = 2.21766
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2003)] = 3.56608
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2004)] = 4.92692
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2005)] = 3.71561
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加共和国') & (dta_droped4.year == 2006)] = 4.28191

dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2000)] = 3.12623
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2001)] = 178.40876
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2002)] = 312.39007
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2003)] = 312.38102
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2004)] = 312.41077
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2005)] = 304.62977
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏丹') & (dta_droped4.year == 2006)] = 286.14017

dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2000)] = 0.07323
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2001)] = 0.07561
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2002)] = 0.07449
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2003)] = 0.06860
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2004)] = 0.06477
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2005)] = 0.06855
dta_droped4['exchange_rate'][(dta_droped4.originn == '拉脱维亚') & (dta_droped4.year == 2006)] = 0.06987

dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2000)] = 1.40238
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2001)] = 5.83914
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2002)] = 7.68471
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2003)] = 6.97692
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2004)] = 7.13262
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2005)] = 8.20172
dta_droped4['exchange_rate'][(dta_droped4.originn == '南斯拉夫联盟共和国') & (dta_droped4.year == 2006)] = 8.70466

dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2000)] = 0.16577
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2001)] = 0.17417
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2002)] = 0.17289
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2003)] = 0.16106
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2004)] = 0.15664
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2005)] = 0.15962
dta_droped4['exchange_rate'][(dta_droped4.originn == '突尼斯') & (dta_droped4.year == 2006)] = 0.16855

dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2000)] = 129.13859
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2001)] = 132.50106
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2002)] = 133.52758
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2003)] = 135.63285
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2004)] = 135.53366
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2005)] = 136.91910
dta_droped4['exchange_rate'][(dta_droped4.originn == '蒙古') & (dta_droped4.year == 2006)] = 142.53918

dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2000)] = 97.07939
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2001)] = 107.24475
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2002)] = 120.25316
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2003)] = 128.55864
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2004)] = 134.62124
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2005)] = 140.58382
dta_droped4['exchange_rate'][(dta_droped4.originn == '坦桑尼亚') & (dta_droped4.year == 2006)] = 161.55840

dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2000)] = 2.05470
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2001)] = 2.11131
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2002)] = 2.00818
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2003)] = 1.67576
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2004)] = 1.52396
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2005)] = 1.53909
dta_droped4['exchange_rate'][(dta_droped4.originn == '爱沙尼亚') & (dta_droped4.year == 2006)] = 1.56618

dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2000)] = 0.06078
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2001)] = 0.07219
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2002)] = 0.14867
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2003)] = 0.15183
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2004)] = 0.16044
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2005)] = 0.16683
dta_droped4['exchange_rate'][(dta_droped4.originn == '利比亚') & (dta_droped4.year == 2006)] = 0.17079

dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2000)] = 5.60359
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2001)] = 5.84713
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2002)] = 5.48099
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2003)] = 4.44751
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2004)] = 3.90657
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2005)] = 3.80031
dta_droped4['exchange_rate'][(dta_droped4.originn == '斯洛伐克共和国') & (dta_droped4.year == 2006)] = 3.72887

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2000)] = 0.04555
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2001)] = 0.04562
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2002)] = 0.04568
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2003)] = 0.04567
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2004)] = 0.04566
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2005)] = 0.04645
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴林') & (dta_droped4.year == 2006)] = 0.04743

dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2000)] = 0.12080
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2001)] = 0.12086
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2002)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2003)] = 0.12096
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2004)] = 0.12097
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2005)] = 0.12221
dta_droped4['exchange_rate'][(dta_droped4.originn == '萨尔瓦多') & (dta_droped4.year == 2006)] = 0.12557

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2000)] = 429.39060
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2001)] = 542.02614
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2002)] = 771.14758
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2003)] = 768.97887
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2004)] = 743.20784
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2005)] = 758.86059
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴拉圭') & (dta_droped4.year == 2006)] = 756.63397

dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2000)] = 5.24531
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2001)] = 5.71349
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2002)] = 6.04935
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2003)] = 7.10126
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2004)] = 7.95155
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2005)] = 8.08769
dta_droped4['exchange_rate'][(dta_droped4.originn == '牙买加') & (dta_droped4.year == 2006)] = 8.09264

dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2000)] = 87.64918
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2001)] = 89.17077
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2002)] = 79.86657
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2003)] = 67.54419
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2004)] = 63.26251
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2005)] = 66.15679
dta_droped4['exchange_rate'][(dta_droped4.originn == '多哥') & (dta_droped4.year == 2006)] = 69.61188

dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2000)] = 0.73005
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2001)] = 2.15730
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2002)] = 4.71778
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2003)] = 7.03643
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2004)] = 7.03714
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2005)] = 10.22125
dta_droped4['exchange_rate'][(dta_droped4.originn == '安哥拉') & (dta_droped4.year == 2006)] = 11.17416

dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2000)] = 0.05379
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2001)] = 0.05473
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2002)] = 0.05190
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2003)] = 0.04829
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2004)] = 0.04130
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2005)] = 0.04345
dta_droped4['exchange_rate'][(dta_droped4.originn == '马耳他') & (dta_droped4.year == 2006)] = 0.04395

dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2000)] = 1.81658
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2001)] = 1.94004
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2002)] = 2.09243
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2003)] = 2.18675
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2004)] = 2.29403
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2005)] = 2.42225
dta_droped4['exchange_rate'][(dta_droped4.originn == '洪都拉斯') & (dta_droped4.year == 2006)] = 2.46121

dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2000)] = 0.75330
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2001)] = 0.74197
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2002)] = 0.73968
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2003)] = 0.74277
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2004)] = 0.75268
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2005)] = 0.77106
dta_droped4['exchange_rate'][(dta_droped4.originn == '特立尼达和多巴哥') & (dta_droped4.year == 2006)] = 0.77452

dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2000)] = 0.04651
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2001)] = 0.04659
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2002)] = 0.04671
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2003)] = 0.04666
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2004)] = 0.04667
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2005)] = 0.04742
dta_droped4['exchange_rate'][(dta_droped4.originn == '阿曼') & (dta_droped4.year == 2006)] = 0.04822

dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2000)] = 87.64918
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2001)] = 89.17077
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2002)] = 79.86657
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2003)] = 67.54419
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2004)] = 63.26251
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2005)] = 66.15679
dta_droped4['exchange_rate'][(dta_droped4.originn == '贝宁') & (dta_droped4.year == 2006)] = 69.61188

dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2000)] = 87.64918
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2001)] = 89.17077
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2002)] = 79.86657
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2003)] = 67.54419
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2004)] = 63.26251
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2005)] = 66.15679
dta_droped4['exchange_rate'][(dta_droped4.originn == '塞内加尔') & (dta_droped4.year == 2006)] = 69.61188

dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2000)] = 87.64918
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2001)] = 89.17077
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2002)] = 79.86657
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2003)] = 67.54419
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2004)] = 63.26251
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2005)] = 66.15679
dta_droped4['exchange_rate'][(dta_droped4.originn == '科特迪瓦共和国') & (dta_droped4.year == 2006)] = 69.61188

dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2000)] = 8.57810
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2001)] = 9.18525
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2002)] = 9.72744
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2003)] = 9.45695
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2004)] = 9.05645
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2005)] = 9.04129
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼泊尔') & (dta_droped4.year == 2006)] = 9.44723

dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2000)] = 0.97613
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2001)] = 1.01763
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2002)] = 1.06322
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2003)] = 1.06305
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2004)] = 1.07545
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2005)] = 1.07928
dta_droped4['exchange_rate'][(dta_droped4.originn == '埃塞俄比亚') & (dta_droped4.year == 2006)] = 1.13301

dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2000)] = 87.64918
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2001)] = 89.17077
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2002)] = 79.86657
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2003)] = 67.54419
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2004)] = 63.26251
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2005)] = 66.15679
dta_droped4['exchange_rate'][(dta_droped4.originn == '喀麦隆') & (dta_droped4.year == 2006)] = 69.61188

dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2000)] = 0.33660
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2001)] = 0.40602
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2002)] = 0.46309
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2003)] = 0.42286
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2004)] = 0.38839
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2005)] = 0.38187
dta_droped4['exchange_rate'][(dta_droped4.originn == '巴布亚新几内亚') & (dta_droped4.year == 2006)] = 0.39025

dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2000)] = 0.12080
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2001)] = 0.12086
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2002)] = 0.24605
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2003)] = 0.51273
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2004)] = 2.54042
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2005)] = 2.56639
dta_droped4['exchange_rate'][(dta_droped4.originn == '古巴') & (dta_droped4.year == 2006)] = 2.85182

dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2000)] = 1.91717
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2001)] = 1.98581
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2002)] = 2.21766
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2003)] = 3.56608
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2004)] = 4.92692
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2005)] = 3.71561
dta_droped4['exchange_rate'][(dta_droped4.originn == '多米尼加') & (dta_droped4.year == 2006)] = 4.28191

dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2000)] = 0.25763
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2001)] = 0.27552
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2002)] = 0.26180
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2003)] = 0.22840
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2004)] = 0.21010
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2005)] = 0.20705
dta_droped4['exchange_rate'][(dta_droped4.originn == '斐济') & (dta_droped4.year == 2006)] = 0.21784

dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2000)] = 1.48284
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2001)] = 1.61222
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2002)] = 1.71830
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2003)] = 1.81118
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2004)] = 1.91808
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2005)] = 2.02466
dta_droped4['exchange_rate'][(dta_droped4.originn == '尼加拉瓜') & (dta_droped4.year == 2006)] = 2.26645

dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2000)] = 0.20830
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2001)] = 0.21522
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2002)] = 0.21704
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2003)] = 0.21236
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2004)] = 0.20584
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2005)] = 0.20547
dta_droped4['exchange_rate'][(dta_droped4.originn == '文莱') & (dta_droped4.year == 2006)] = 0.20239

dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2000)] = 202.04748
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2001)] = 234.53075
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2002)] = 246.18956
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2003)] = 246.93518
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2004)] = 277.24012
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2005)] = 445.51699
dta_droped4['exchange_rate'][(dta_droped4.originn == '几内亚') & (dta_droped4.year == 2006)] = 636.01095

dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2000)] = 1.62433
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2001)] = 1.99118
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2002)] = 2.45138
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2003)] = 3.29444
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2004)] = 3.60492
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2005)] = 3.57271
dta_droped4['exchange_rate'][(dta_droped4.originn == '冈比亚') & (dta_droped4.year == 2006)] = 3.67267

dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2000)] = 9.53036
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2001)] = 11.80675
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2002)] = 11.08864
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2003)] = 9.28739
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2004)] = 8.49969
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2005)] = 7.68888
dta_droped4['exchange_rate'][(dta_droped4.originn == '冰岛') & (dta_droped4.year == 2006)] = 8.80330

dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2000)] = 233.71278
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2001)] = 372.28532
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2002)] = 394.49410
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2003)] = 391.50716
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2004)] = 230.68840
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2005)] = 183.99454
dta_droped4['exchange_rate'][(dta_droped4.originn == '伊拉克') & (dta_droped4.year == 2006)] = 191.64607

dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2000)] = 910.41513
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2001)] = 930.70318
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2002)] = 955.89576
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2003)] = 963.43172
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2004)] = 982.90337
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2005)] = 1304.42561
dta_droped4['exchange_rate'][(dta_droped4.originn == '老挝') & (dta_droped4.year == 2006)] = 1323.02433

dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2000)] = 5.87771
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2001)] = 6.83567
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2002)] = 6.90662
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2003)] = 128.09708
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2004)] = 650.02706
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2005)] = 5460.70214
dta_droped4['exchange_rate'][(dta_droped4.originn == '津巴布韦') & (dta_droped4.year == 2006)] = 12399.04667

dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2000)] = 105.88859
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2001)] = 171.10789
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2002)] = 266.07985
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2003)] = 302.69549
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2004)] = 302.72615
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2005)] = 322.10549
dta_droped4['exchange_rate'][(dta_droped4.originn == '苏里南') & (dta_droped4.year == 2006)] = 347.26937

dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2000)] = 0.13106
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2001)] = 0.13499
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2002)] = 0.12835
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2003)] = 0.10710
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2004)] = 0.09740
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2005)] = 0.09835
dta_droped4['exchange_rate'][(dta_droped4.originn == '挪威') & (dta_droped4.year == 2006)] = 0.10006

dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2000)] = 0.26575
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2001)] = 0.26588
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2002)] = 0.26612
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2003)] = 0.26611
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2004)] = 1.98543
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2005)] = 20.77551
dta_droped4['exchange_rate'][(dta_droped4.originn == '朝鲜') & (dta_droped4.year == 2006)] = 17.70492